# Entorno de experimentación `Linear Chain CRF`

Entorno dónde se recoge toda la información del lenguaje en la construcción de las *feature lists*

### Parámetros generales

* Maximo Iteraciones = 50
* K = 3

### Parametros por modelo

* `linearCRF_reg.crfsuite`
    * l1 = 0.1
    * l2 = 0.001
* `linearCRF_noreg.crfsuite`
    * l1 = 0
    * l2 = 0
* `linearCRF_l1_zero.crfsuite`
    * l1 = 0
    * l2 = 0.001
* `linearCRF_l2_zero.crfsuite`
    * l1 = 0.1
    * l2 = 0



## Importando bibliotecas de python

In [1]:
import os
import sys  
import random
import time
import pycrfsuite
import numpy as np
from sklearn.model_selection import KFold
from utils import (get_corpus, WordsToLetter, accuracy_score, model_trainer,
                   model_tester, write_report, eval_labeled_positions,
                   bio_classification_report)

## Funciones auxiliares    

In [19]:
def get_feature_lists(sent):
    ''' Reglas que configuran las feature lists para entrenamiento

    :param sent: Data as `[[[[[letter, POS, BIO-label],...],words],sents]]`
    :type: list
    :return: list of words with characters as features list:
        [[[[[letterfeatures],POS,BIO-label],letters],words]]
    :rtype: list
    '''

    featurelist = []
    senlen = len(sent)
    # each word in a sentence
    for i in range(senlen):
        word = sent[i]
        wordlen = len(word)
        lettersequence = ''
        # each letter in a word
        for j in range(wordlen):
            letter = word[j][0]
            # gathering previous letters
            lettersequence += letter
            # ignore digits
            if not letter.isdigit():
                features = [
                    'bias',
                    'letterLowercase=' + letter.lower(),
                ]
                # Position of word in sentence
                if i == senlen -1:
                    features.append("EOS")
                else:
                    features.append("BOS")

                # Pos tag sequence (Don't get pos tag if sentence is 1 word long)
                if i > 0 and senlen > 1:
                    features.append('prevpostag=' + sent[i-1][0][1])
                    if i != senlen-1:
                        features.append('nxtpostag=' + sent[i+1][0][1])

                # Position of letter in word
                if j == 0:
                    features.append('BOW')
                elif j == wordlen-1:
                    features.append('EOW')
                else:
                    features.append('letterposition=-%s' % str(wordlen-1-j))

                # Letter sequences before letter
                if j >= 4:
                    features.append('prev4letters=' + lettersequence[j-4:j].lower() + '>')
                if j >= 3:
                    features.append('prev3letters=' + lettersequence[j-3:j].lower() + '>')
                if j >= 2:
                    features.append('prev2letters=' + lettersequence[j-2:j].lower() + '>')
                if j >= 1:
                    features.append('prevletter=' + lettersequence[j-1:j].lower() + '>')

                # letter sequences after letter
                if j <= wordlen-2:
                    nxtlets = word[j+1][0]
                    features.append('nxtletter=<' + nxtlets.lower())
                if j <= wordlen-3:
                    nxtlets += word[j+2][0]
                    features.append('nxt2letters=<' + nxtlets.lower())
                if j <= wordlen-4:
                    nxtlets += word[j+3][0]
                    features.append('nxt3letters=<' + nxtlets.lower())
                if j <= wordlen-5:
                    nxtlets += word[j+4][0]
                    features.append('nxt4letters=<' + nxtlets.lower())
            featurelist.append(features)
    return featurelist


def get_labels(sent, flag=0):
    labels = []
    for word in sent:
        for letter in word:
            labels.append(letter[2])
    return labels


def sent2features(data):
    return [get_feature_lists(sent) for sent in data]


def sent2labels(data):
    return [get_labels(sent) for sent in data]

### Funciones de Train y Test 

In [3]:
def model_trainer(train_data, hyper):
    """ Entrena un modelo y lo guarda en disco

    Función encargada de entrenar un modelo con base en los hyperparametro y
    lo guarda como un archivo utilizable por `pycrfsuite`

    Parameters
    ----------
    train_data : list
    models_path : str
    hyper : dict
    verbose : bool
    k : int

    Returns
    -------
    train_time : float
        Tiempo de entrenamiento
    compositive_name : str
        Nombre del modelo entrenado
    """
    X_train = sent2features(train_data)
    y_train = sent2labels(train_data)
    
    # Train the model
    trainer = pycrfsuite.Trainer(verbose=True)

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq)

    # Set training parameters. L-BFGS is default. Using Elastic Net (L1 + L2)
    trainer.set_params({
            'c1': hyper['L1'],  # coefficient for L1 penalty
            'c2': hyper['L2'],  # coefficient for L2 penalty
            'max_iterations': hyper['max-iter']  # early stopping
        })
    # The program saves the trained model to a file:
    start = time.time()
    trainer.train(hyper['path'])
    end = time.time()
    train_time = end - start
    return train_time


def model_tester(test_data, model_path):
    """ Prueba un modelo preentrenado

    Recibe los datos de prueba y realiza las pruebas con el modelo previo

    Parameters
    ----------
    test_data : list
    models_path : str
    model_name : str
    verbose : bool

    Returns
    -------
    y_test : list
        Etiquetas reales
    y_pred : list
        Etiquetas predichas por el modelo
    tagger : Object
        Objeto que etiqueta con base en el modelo
    """
    X_test = sent2features(test_data)
    y_test = sent2labels(test_data)

    # ### Make Predictions
    tagger = pycrfsuite.Tagger()
    # Passing model to tagger
    tagger.open(model_path)  
    # Tagging task using the model
    y_pred = [tagger.tag(xseq) for xseq in X_test]
    # Closing tagger
    tagger.close()
    return y_test, y_pred

## Obteniendo corpus completo

In [4]:
corpus = get_corpus('corpus_otomi_mod', '../corpora/') + \
         get_corpus('corpus_otomi_hard', '../corpora/')
letter_corpus = WordsToLetter(corpus)
dataset = np.array(letter_corpus, dtype=object)

## Parametros base

In [5]:
models_path = 'models/'
env_name = "linearCRF"
max_iter = 50
k = 3
kf = KFold(n_splits=k, shuffle=True)

## Parámetros para `linearCRF_reg.crfsuite`

In [6]:
params = {"L1": 0.1, "L2": 1e-3, "max-iter": max_iter}
variant = "reg"

### Entrenamiento y Tests

In [10]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'linearCRF_reg_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28124
Seconds required: 0.059

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 125339.680931
Feature norm: 1.000000
Error norm: 23323.612616
Active features: 28015
Line search trials: 1
Line search step: 0.000042
Seconds required for this iteration: 4.028

***** Iteration #2 *****
Loss: 94852.692252
Feature norm: 5.436096
Error norm: 27513.620756
Active features: 27968
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 9.6

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28053
Seconds required: 0.054

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 126595.858961
Feature norm: 1.000000
Error norm: 23397.752985
Active features: 27973
Line search trials: 1
Line search step: 0.000042
Seconds required for this iteration: 4.026

***** Iteration #2 *****
Loss: 96987.359359
Feature norm: 5.555334
Error norm: 27799.726616
Active features: 27898
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 9.974

***** Iteration #3 *****
Loss: 66180.554088
Feature norm: 4.454247
Error norm: 6299.586104
Active features: 26918
Line search trials: 1
Line search step: 1.000000
Seconds required for 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27604
Seconds required: 0.051

L-BFGS optimization
c1: 0.100000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 122042.872456
Feature norm: 1.000000
Error norm: 22175.441354
Active features: 27509
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 3.827

***** Iteration #2 *****
Loss: 93320.103971
Feature norm: 5.504269
Error norm: 26868.122114
Active features: 27471
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 9.641

***** Iteration #3 *****
Loss: 64771.448480
Feature norm: 4.409146
Error norm: 6485.277230
Active features: 26409
Line search trials: 1
Line search step: 1.000000
Seconds required for 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `linearCRF_noreg.crfsuite`

In [13]:
params = {"L1": 0.0, "L2": 0.0, "max-iter": max_iter}
variant = "noreg"

### Entrenamiento y Tests

In [14]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'linearCRF_noreg_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27421
Seconds required: 0.053

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 94632.299530
Feature norm: 5.000000
Error norm: 26332.478035
Active features: 27421
Line search trials: 2
Line search step: 0.000212
Seconds required for this iteration: 5.515

***** Iteration #2 *****
Loss: 73449.149412
Feature norm: 3.714323
Error norm: 12596.647825
Active features: 27421
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28046
Seconds required: 0.055

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 92394.859832
Feature norm: 5.000000
Error norm: 25765.328428
Active features: 28046
Line search trials: 2
Line search step: 0.000215
Seconds required for this iteration: 5.786

***** Iteration #2 *****
Loss: 72260.095328
Feature norm: 3.751747
Error norm: 12575.566974
Active features: 28046
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.012

***** Iteration #3 *****
Loss: 65365.298400
Feature norm: 4.710026
Error norm: 4929.722290
Active features: 28046
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28082
Seconds required: 0.062

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 93656.607073
Feature norm: 5.000000
Error norm: 25761.526069
Active features: 28082
Line search trials: 2
Line search step: 0.000215
Seconds required for this iteration: 6.794

***** Iteration #2 *****
Loss: 74274.057530
Feature norm: 3.731300
Error norm: 13104.343145
Active features: 28082
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.128

***** Iteration #3 *****
Loss: 66856.371062
Feature norm: 4.704276
Error norm: 5066.979079
Active features: 28082
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `linearCRF_l1_zero.crfsuite`

In [15]:
params = {"L1": 0.0, "L2": 1e-3, "max-iter": max_iter}
variant = "l1_zero"

### Entrenamiento y Tests

In [16]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'linearCRF_l1_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27570
Seconds required: 0.065

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 92499.549456
Feature norm: 5.000000
Error norm: 25580.705844
Active features: 27570
Line search trials: 2
Line search step: 0.000215
Seconds required for this iteration: 6.360

***** Iteration #2 *****
Loss: 72968.185262
Feature norm: 3.755774
Error norm: 12775.296266
Active features: 27570
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28188
Seconds required: 0.056

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 94023.738716
Feature norm: 5.000000
Error norm: 26202.174724
Active features: 28188
Line search trials: 2
Line search step: 0.000213
Seconds required for this iteration: 5.640

***** Iteration #2 *****
Loss: 73380.273210
Feature norm: 3.735487
Error norm: 12650.625720
Active features: 28188
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 1.865

***** Iteration #3 *****
Loss: 66482.303404
Feature norm: 4.688695
Error norm: 4997.955573
Active features: 28188
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 28018
Seconds required: 0.053

L-BFGS optimization
c1: 0.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 94180.368005
Feature norm: 5.000000
Error norm: 25967.590445
Active features: 28018
Line search trials: 2
Line search step: 0.000214
Seconds required for this iteration: 6.166

***** Iteration #2 *****
Loss: 74270.316704
Feature norm: 3.715768
Error norm: 13139.101692
Active features: 28018
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 2.136

***** Iteration #3 *****
Loss: 66883.509587
Feature norm: 4.683041
Error norm: 5105.313423
Active features: 28018
Line search trials: 1
Line search step: 1.000000
Seconds required for t

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Parámetros para `linearCRF_l2_zero.crfsuite`

In [17]:
params = {"L1": 0.1, "L2": 0.0, "max-iter": max_iter}
variant = "l2_zero"

### Entrenamiento y Tests

In [18]:
%%time
i = 0
full_time = 0
accuracy_set = []
for train_index, test_index in kf.split(dataset):
    i += 1
    train_data, test_data = dataset[train_index], dataset[test_index]
    model_name = f"{env_name}_{variant}_k_{i}.crf"
    params['path'] = os.path.join(models_path, env_name, model_name)
    print("*"*50)
    print(f"Entrenando nuevo modelo '{model_name}' | K = {i}")
    print(f"len train: {len(train_data)} len test: {len(test_data)}")
    print("*"*50)
    train_time = model_trainer(train_data, params)
    full_time += train_time
    print("*"*50)
    print(f"Tiempo de entrenamiento: {train_time}[s] | {train_time / 60}[m]")
    print("Test del modelo")
    y_test, y_pred = model_tester(test_data, params['path'])
    accuracy_set.append(accuracy_score(y_test, y_pred))
    print(f"Partial accuracy: {accuracy_set[i - 1]}\n")
    # Reports
    eval_labeled_positions(y_test, y_pred)
    print(bio_classification_report(y_test, y_pred))
print("\n\nAccuracy Set -->", accuracy_set)
train_time_format = str(round(full_time / 60, 2)) + "[m]"
print(f"\nTime>> {train_time_format}")
train_size = len(train_data)
test_size = len(test_data)
params['k-folds'] = k
write_report(model_name, train_size, test_size, accuracy_set, train_time_format,
             params)

**************************************************
Entrenando nuevo modelo 'linearCRF_l2_zero_k_1.crf' | K = 1
len train: 1179 len test: 590
**************************************************
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27943
Seconds required: 0.061

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 126657.680848
Feature norm: 1.000000
Error norm: 23324.370310
Active features: 27821
Line search trials: 1
Line search step: 0.000042
Seconds required for this iteration: 4.536

***** Iteration #2 *****
Loss: 96854.859930
Feature norm: 5.534877
Error norm: 27850.084263
Active features: 27761
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration:

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27888
Seconds required: 0.072

L-BFGS optimization
c1: 0.100000
c2: 0.000000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 121831.047376
Feature norm: 1.000000
Error norm: 22833.661233
Active features: 27776
Line search trials: 1
Line search step: 0.000043
Seconds required for this iteration: 3.826

***** Iteration #2 *****
Loss: 90787.463966
Feature norm: 5.311965
Error norm: 26660.041106
Active features: 27724
Line search trials: 5
Line search step: 0.062500
Seconds required for this iteration: 9.657

***** Iteration #3 *****
Loss: 64697.215883
Feature norm: 4.273136
Error norm: 7119.116445
Active features: 26703
Line search trials: 1
Line search step: 1.000000
Seconds required for 

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

         B-1.cnt       1.00      1.00      1.00        24
         I-1.cnt       1.00      1.00      1.00        48
         B-1.cpl       1.00      1.00      1.00        64
         I-1.cpl       1.00      1.00      1.00        64
     B-1.cpl.irr       1.00      1.00      1.00         4
     I-1.cpl.irr       1.00      1.00      1.00         4
         B-1.enf       1.00      0.43      0.60         7
         I-1.enf       1.00      0.43      0.60         7
         B-1.icp       0.99      1.00      0.99        98
         I-1.icp       0.99      1.00      0.99        98
     B-1.icp.irr       1.00      1.00      1.00         2
     I-1.icp.irr       1.00      1.00      1.00         4
         B-1.obj       0.93      0.90      0.91        41
         I-1.obj       0.93      0.90      0.91        41
         B-1.pls       0.00      0.00      0.00         3
         I-1.pls       0.00      0.00      0.00         3
         B-1.

/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umoqnier/.pyenv/versions/tesis-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
